In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="qpeexact",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------",qc.name," with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

--------------- qpeexact  with qubit 3 ---------------
--------------- qpeexact  with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7da7af656fc0>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7da7b02b38c0>)]


In [2]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

simulator = AerSimulator()

jobs = []
for qubit_number, circuit_object in circuits:
    # Run noisy simulation (counts)
    job = simulator.run(circuit_object, shots=1000)
    results = job.result()
    counts = results.get_counts()

    # Get ideal distribution from statevector
    qc_nom = circuit_object.remove_final_measurements(inplace=False)
    state = Statevector.from_instruction(qc_nom)
    ideal_probs = state.probabilities_dict()

    jobs.append((qubit_number, counts, ideal_probs))

print("Collected results:")
for res in jobs:
    print(res)


Collected results:
(3, {'10': 1000}, {'000': 2.3496345321524272e-32, '001': 7.28272657618405e-34, '010': 9.849226832859281e-35, '011': 6.92211582125373e-33, '100': 4.5066760698661324e-32, '101': 3.5993726390809435e-32, '110': 1.0, '111': 5.097994436730214e-32})
(4, {'100': 1000}, {'0000': 5.007417855406809e-33, '0001': 9.360873108461332e-33, '0010': 7.695773248803968e-35, '0011': 8.982072257504544e-33, '0100': 1.1759678369401415e-32, '0101': 1.0171922099185184e-32, '0110': 1.4644948990741023e-33, '0111': 3.440555763175674e-34, '1000': 1.20563214518641e-31, '1001': 8.587745149029165e-32, '1010': 1.3239799405325488e-32, '1011': 6.865995209303065e-32, '1100': 0.9999999999999993, '1101': 1.1774717985749911e-32, '1110': 1.260467218640668e-32, '1111': 1.98841930970151e-32})


In [3]:
# Hellinger fidelity function
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (probabilities)."""
    p_sum = sum(p.values())
    p_norm = {k: v / p_sum for k, v in p.items()}

    q_sum = sum(q.values())
    q_norm = {k: v / q_sum for k, v in q.items()}

    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

In [5]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (qpe-exact, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (qpe-exact, 3 qubits): 0.0000
Hellinger Fidelity (qpe-exact, 4 qubits): 0.0000
